In [5]:
from BucketSort import BucketSort
import random

import random

def generate_connected_graph(n, extra_edges, max_weight):
    """
    Generate a connected undirected graph with n nodes.

    Parameters:
        n           : number of vertices (int)
        extra_edges : number of additional random edges (int)
        max_weight  : maximum weight for any edge (int)

    Returns:
        List of edges in {(u, v): weight} format
    """
    edges = []
    connected = set([0])

    # Ensure connectivity by building a spanning tree
    for i in range(1, n):
        u = i
        v = random.choice(list(connected))
        weight = random.randint(1, max_weight)
        edge_key = (min(u, v), max(u, v))
        edges.append({edge_key: weight})
        connected.add(u)

    # Track existing edges for duplicate checking
    existing_edges = set()
    for edge in edges:
        u, v = list(edge.keys())[0]
        edge_pair = (min(u, v), max(u, v))
        existing_edges.add(edge_pair)

    # Add extra edges
    added_edges = 0
    while added_edges < extra_edges:
        u, v = random.sample(range(n), 2)
        edge_pair = (min(u, v), max(u, v))

        if edge_pair in existing_edges:
            continue

        weight = random.randint(1, max_weight)
        edges.append({edge_pair: weight})
        existing_edges.add(edge_pair)
        added_edges += 1

    return edges

# test example
m = 999
A = generate_connected_graph(1000, 10000, m)

BucketSort(A, m)

[{(57, 864): 1},
 {(201, 940): 1},
 {(188, 595): 1},
 {(193, 204): 1},
 {(153, 626): 1},
 {(328, 974): 1},
 {(206, 798): 1},
 {(548, 705): 1},
 {(439, 485): 2},
 {(263, 429): 2},
 {(107, 202): 2},
 {(184, 253): 2},
 {(262, 544): 2},
 {(47, 831): 2},
 {(694, 910): 2},
 {(230, 584): 2},
 {(402, 731): 3},
 {(250, 931): 3},
 {(294, 559): 3},
 {(132, 778): 3},
 {(67, 569): 3},
 {(139, 639): 3},
 {(219, 314): 4},
 {(400, 910): 4},
 {(700, 918): 4},
 {(854, 868): 4},
 {(45, 454): 4},
 {(151, 653): 4},
 {(783, 911): 4},
 {(664, 898): 5},
 {(170, 702): 5},
 {(23, 134): 5},
 {(327, 896): 5},
 {(227, 786): 5},
 {(248, 605): 5},
 {(641, 845): 5},
 {(25, 33): 5},
 {(41, 118): 6},
 {(254, 961): 6},
 {(61, 912): 6},
 {(360, 825): 6},
 {(416, 417): 6},
 {(803, 965): 6},
 {(224, 714): 6},
 {(550, 830): 7},
 {(339, 521): 7},
 {(16, 563): 7},
 {(509, 977): 7},
 {(545, 748): 7},
 {(479, 691): 7},
 {(375, 389): 8},
 {(647, 979): 8},
 {(528, 882): 8},
 {(533, 593): 8},
 {(161, 617): 8},
 {(40, 456): 8},
 {(